In [ ]:
import os
import sys

sys.path.append(os.path.abspath("src/"))

import logging
import time
from pathlib import Path

import hydra
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import torch
import yaml
from giskard import Dataset, Model, scan, testing
from hydra import initialize_config_dir
from joblib import Memory
from omegaconf import DictConfig, OmegaConf
from sklearn.metrics import accuracy_score
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchFastText.datasets import FastTextModelDataset

from src.datasets import SoftClassifDataset, TextClassificationDataModule
from src.utils.evaluation import get_inv_mapping

%load_ext autoreload
%autoreload 2

In [ ]:
import os

import mlflow

os.environ['MLFLOW_TRACKING_URI'] = "https://projet-ape-mlflow.user.lab.sspcloud.fr/" 
model_name = "FastText-pytorch-2025"
module = mlflow.pytorch.load_model(f"models:/{model_name}/2")

In [ ]:
import pandas as pd

input_data = pd.DataFrame([
    {
        "description_activity": "Coiffure",
        "other_nature_activity": None,
        "precision_act_sec_agricole": None,
        "type_form": None,
        "nature": None,
        "surface": None,
        "cj": None,
        "activity_permanence_status": None
    }
])
# Create params dictionary
params_dict = {
    "nb_echos_max": 5,
    "prob_min": 0.00,
    "dataloader_params":
    {
        "pin_memory": False,
        "persistent_workers": False,
        "num_workers": 0,
        "batch_size": 1,
    }
}

module.predict(input_data, params=params_dict)  # Merge with run parameters